In [1]:
import pygad
import numpy as np
import matplotlib.pyplot as plt
import json
import pandas as pd

In [2]:
# Генерация случайных данных для 100 транспортных средств
vehicle_classes = np.random.choice(['A', 'B', 'C', 'D', 'E', 'F'], size=6)
imo = np.random.randint(low=9000000, high=9999999, size=6)
speeds = np.random.randint(low=0, high=20, size=6)
start_times = np.random.randint(low=0, high=30, size=6)
end_times = start_times + np.random.randint(low=1, high=30, size=6)
start_points = np.random.randint(low=0, high=3, size=6)
end_points = np.random.randint(low=0, high=4, size=6)

# Создание датасета с сгенерированными данными
vehicle_data = pd.DataFrame({
    'class': vehicle_classes,
    'imo': imo,
    'speed': speeds,
    'start_time': start_times,
    'end_time': end_times,
    'start_point': start_points,
    'end_point': end_points
})

# Сохранение датасета в формате CSV
#vehicle_data.to_csv('vehicles.csv', index=False)

In [3]:
# Чтение датасета с информацией о транспортных средствах
# Чтение датасета с информацией о транспортных средствах
"""vehicle_data = pd.read_csv('vehicles.csv')

# Создание словаря с информацией о скоростях, рекомендуемых временах, начальных и конечных точках транспортных средств
vehicle_speeds = {}
recommended_start_times = {}
recommended_end_times = {}
start_points = {}
end_points = {}
for index, row in vehicle_data.iterrows():
    vehicle_class = row['class']
    imo_value = row['imo']
    speed = row['speed']
    start_time = row['start_time']
    end_time = row['end_time']
    start_point = row['start_point']
    end_point = row['end_point']
    vehicle_speeds[vehicle_class] = speed
    recommended_start_times[vehicle_class] = start_time
    recommended_end_times[vehicle_class] = end_time
    start_points[vehicle_class] = start_point
    end_points[vehicle_class] = end_point"""

vehicle_speeds = {}
recommended_start_times = {}
recommended_end_times = {}
start_points = {}
end_points = {}
for index, row in vehicle_data.iterrows():
    vehicle_class = row['class']
    imo_value = row['imo']
    speed = row['speed']
    start_time = row['start_time']
    end_time = row['end_time']
    start_point = row['start_point']
    end_point = row['end_point']
    vehicle_speeds[vehicle_class] = speed
    recommended_start_times[vehicle_class] = start_time
    recommended_end_times[vehicle_class] = end_time
    start_points[vehicle_class] = start_point
    end_points[vehicle_class] = end_point

In [4]:
num_routes = len(start_points)
eta_values = 2
print(num_routes)
print(start_points)

5
{'D': 2, 'F': 1, 'B': 2, 'C': 2, 'E': 2}


In [5]:
def fitness_function(solution, solution_idx):
    # Разделение генетической популяции на маршруты
    routes = np.split(solution, num_routes, axis=0)
    print(routes)
    # Вычисление значения целевой функции для каждого маршрута
    fitness_values = []
    for route in routes:
        # Добавление начальной точки в маршрут
        print(route)
        route = np.insert(route, 0, start_points[solution_idx])

        # Вычисление значения первого критерия - суммарного отклонения от ETA
        eta_deviations = np.abs(route - eta_values)
        eta_deviation_sum = np.sum(eta_deviations)

        # Вычисление значения второго критерия - суммарного пройденного расстояния
        distance_sum = np.sum(route)

        # Вычисление значения третьего критерия - потери скорости движения
        speed_loss = 0
        for i in range(1, len(route)):
            # Определение класса транспортного средства на основе данных из датасета
            vehicle_class = vehicle_data.loc[solution_idx, 'class']

            # Проверка возможности пройти по маршруту выше классом сплоченности
            if route[i] > route[i-1]:
                # Проверяем наличие судна соответствующего класса в караване
                caravan = route[:i]
                if vehicle_class in caravan.values():
                    continue  # Можем двигаться выше классом сплоченности

            # Вычисление потери скорости на основе класса транспортного средства
            speed = vehicle_speeds[vehicle_class]
            distance = route[i] - route[i-1]
            speed_loss += (1 - (speed / 100)) * distance

        # Учет рекомендуемого времени начала и окончания движения
        start_time = recommended_start_times[vehicle_class]
        end_time = recommended_end_times[vehicle_class]
        time_loss = 0

        # Проверка, находится ли маршрут вне рекомендуемого временного интервала
        if route[1] < start_time or route[-1] > end_time:
            time_loss = 100  # Высокая штрафная функция для маршрутов, не соответствующих рекомендуемым временам

        # Вычисление значения целевой функции
        fitness_value = eta_deviation_sum + distance_sum + speed_loss + time_loss

        # Добавление значения целевой функции в список
        fitness_values.append(fitness_value)

    # Выбор одного значения из списка, например, среднего значения
    fitness_value = np.mean(fitness_values)

    # Возвращение значения целевой функции
    return fitness_value

In [6]:
# Создание экземпляра генетического алгоритма
ga_instance = pygad.GA(
    num_generations=50,  # Количество поколений
    num_parents_mating=10,  # Количество родительских пар для спаривания
    fitness_func=fitness_function,  # Функция приспособленности
    sol_per_pop=50,
    num_genes=num_routes,
    mutation_type=None
)

In [7]:
# Запуск генетического алгоритма
ga_instance.run()

[array([-1.57268169]), array([3.72365423]), array([-3.45811734]), array([-0.94017866]), array([-3.06785326])]
[-1.57268169]


KeyError: 0

In [ ]:
# Получение лучших решений для каждого маршрута
best_solutions = ga_instance.best_solution()
best_fitness = ga_instance.best_solution_fitness()

print("Лучшее решение: ", best_solutions)
print("Значение целевой функции для лучшего решения: ", best_fitness)

In [ ]:
# Визуализация результатов
fig, ax = plt.subplots()

for i in range(num_routes):
    start_times = np.cumsum(best_solutions[i])
    task_lengths = [task_length for task_length in best_solutions[i]]
    ax.broken_barh([(start_times[j], task_lengths[j]) for j in range(len(start_times))], (10*i, 9))

ax.set_xlabel('Время')
ax.set_ylabel('Маршрут')
ax.set_title('Диаграмма Ганта')
ax.set_yticks([10*i+5 for i in range(num_routes)])
ax.set_yticklabels([f'Маршрут {i+1}' for i in range(num_routes)])

plt.show()

In [ ]:
# Формирование данных для JSON
data = {
    "best_solutions": best_solutions,
    "best_fitness": best_fitness
}

# Сериализация данных в JSON-строку
json_data = json.dumps(data)

# Отправка JSON-строки
# ...